In [8]:
############### Set working directory to parent directory
import os
os.getcwd()
# os.chdir('F:\\github\\narrative_conservatism\\code')

'F:\\github\\narrative_conservatism\\code'

In [1]:
from bs4 import BeautifulSoup
import requests
import sys

# Access search page ############### loop over all companies?##########################
cik = '0000051143'
type = '10-Q'
# dateb = 20160101; find filings prior to the date 2016y01m01d

# Obtain HTML for search page
# base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}"
# edgar_resp = requests.get(base_url.format(cik, type, dateb))
edgar_resp = requests.get(base_url.format(cik, type))
edgar_str = edgar_resp.text

In [2]:
############### Find the document link
doc_link = list()
soup = BeautifulSoup(edgar_str, 'html.parser')
table_tag = soup.find('table', class_='tableFile2')
rows = table_tag.find_all('tr')

for row in rows[1:]:
    cells = row.find_all('td')
#    if len(cells) > 3:
#        if '2015' in cells[3].text: ##find filings of particular year
    doc_link.append('https://www.sec.gov' + cells[1].a['href'])

In [3]:
############### Save HTML for document page, and extract file identification info
import re

html_link = list()
filenum = list()
fd = list()
rp = list()
name = list()
cik = list()
sic = list()
filetype = list()
fye = list()
state = list()
bazip = list()

for doc in doc_link:
    doc_resp = requests.get(doc)
    doc_str = doc_resp.text
# Save the HTML link
    soup = BeautifulSoup(doc_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile', summary='Document Format Files')
    rows = table_tag.find_all('tr')
    cells = rows[1].find_all('td')
    html_link.append('https://www.sec.gov' + cells[2].a['href'])
# Save the SEC filing number
    div_tag = soup.find('div', id='formHeader')
    secnum = div_tag.find('div', id='secNum')
    a = secnum.get_text().split()[3]
    filenum.append(a)
# Save the Filing Date and Reporting Period
    div_tag = soup.find('div', class_='formContent')
    dates = div_tag.find_all('div', class_='info')
    # Filing Date
    a = dates[0].get_text()
    fd.append(a)
    # Reporting Period
    b = dates[3].get_text()
    rp.append(b)
# Save the Company name and CIK
    div_tag = soup.find('div', class_='companyInfo')
    comname = div_tag.find('span', class_='companyName')
    # Company Name
    a = comname.get_text().split("\n")[0]
    name.append(a)
    # CIK
    b = comname.get_text().split("\n")[1]
    cik.append(b)
# Save Business Address ZIP 
    div_tag = soup.find_all('div', class_='mailer')[1]
    ba = div_tag.get_text().split()
    for w in ba:
        match = str(re.search(r'\d*5', w))
        if match != "None":
            a = ba[ba.index(w)]
            bazip.append(a)
# Save SIC, File Type, Fiscal Year End and State of Incorporation
    div_tag = soup.find('div', class_='companyInfo')
    filinginfo = div_tag.find('p', class_='identInfo')
    # SIC
    a = filinginfo.get_text().split("|")[5].split("SIC")[1].split()[1]
    sic.append(a)
    # File Type
    b = filinginfo.get_text().split("|")[2].split("Type")[1].split(":")[1]
    filetype.append(b)
    # Fiscal Year End
    c = filinginfo.get_text().split("|")[2].split("Type")[0].split(":")[1]
    fye.append(c)
    # State
    d = filinginfo.get_text().split("|")[1].split(":")[1]
    state.append(d)
    
html_link = [w.replace('ix?doc=/', '') for w in html_link]
name = [w.replace(' (Filer)', '') for w in name]
cik = [w.replace('CIK: ', '') for w in cik]
cik = [w.replace(' (see all company filings)', '') for w in cik]

In [6]:
############### Save HTML into TXT files
import urllib.request

for link in html_link:
    urllib.request.urlretrieve(link, '..\\filings\\raw\\'+str(filenum[html_link.index(link)])+'.txt')

In [5]:
############ USE SEC_EDGAR_DOWNLOADER TO DOWNLOAD SEC FILES
#### BUT IT DOWNLOADS ***TXT FILES*** NOT ***HTML***
# from sec_edgar_downloader import Downloader

## Initialize a downloader instance.
## If no argument is passed to the constructor, the package
## will attempt to locate the user's downloads folder.
# dl = Downloader("filings/raw")

# Get all 10-Q filings for Apple
# dl.get("10-Q", "AAPL")

In [11]:
############### Create Data Frame
import pandas as pd

d = {'cik': cik, 'name': name, 'fd': fd, 'rp': rp, 'filenum': filenum, 'bazip': bazip}
df = pd.DataFrame(data=d)
df

,cik,name,fd,rp,filenum,bazip
0,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2019-10-29,2019-09-30,0001558370-19-009324,10504
1,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2019-07-30,2019-06-30,0001558370-19-006560,10504
2,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2019-04-30,2019-03-31,0001104659-19-025328,10504
3,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2018-10-30,2018-09-30,0001104659-18-064673,10504
4,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2018-07-31,2018-06-30,0001104659-18-048404,10504
5,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2018-04-24,2018-03-31,0001104659-18-026178,10504
6,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2017-10-31,2017-09-30,0001104659-17-064968,10504
7,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2017-07-25,2017-06-30,0001104659-17-046808,10504
8,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2017-04-25,2017-03-31,0001104659-17-025881,10504
9,0000051143,INTERNATIONAL BUSINESS MACHINES CORP,2016-10-25,2016-09-30,0001104659-16-151931,10504
